In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange

In [4]:
try:
    from .modules import Model_1D, Model_2D, Model_Text
except ImportError:
    from modules import Model_1D, Model_2D, Model_Text

/home/ay/anaconda3/envs/audio_df/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class MultiView3DModel(nn.Module):
    def __init__(self, cfg=None, args=None, **kwargs):
        super().__init__()
        self.cfg = cfg
        self.args = args
        
        self.model_1d = Model_1D()
        self.model_2d = Model_2D()
        self.model_text = Model_Text()
        
        
        self.classifier = nn.Linear(768, 1)
        
    def forward(self, x: torch.Tensor):
        
        x_1d = self.model_1d(x)
        x_2d = self.model_2d(x)
        x_2d = rearrange(x_2d, "b c h w -> b c (h w)")
        x_2d = F.adaptive_avg_pool1d(x_2d, x_1d.shape[-1])
        x_text = self.model_text(x)
        
        # print(x_1d.shape, x_2d.shape, x_text.shape)
        # return x_1d, x_2d, x_text
        
        feat = (x_1d + x_2d + x_text) / 3
        
        feat = feat.mean(dim=-1)
        logit = self.classifier(feat)
        
        return logit

In [6]:
model = MultiView3DModel()
x = torch.randn(2, 16_000*4)
model(x)

Some weights of the model checkpoint at /usr/local/ay_data/0-model_weights/microsoft_wavlm-base were not used when initializing WavLMModel: ['encoder.pos_conv_embed.conv.weight_v', 'encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing WavLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMModel were not initialized from the model checkpoint at /usr/local/ay_data/0-model_weights/microsoft_wavlm-base and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this mo

tensor([[0.1685],
        [0.1609]], grad_fn=<AddmmBackward0>)